In [1]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/neccam/slt.git

%cd slt

!chmod +x data/download.sh
!data/download.sh

!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

!pip -q install -r requirements.txt

Mounted at /content/drive
Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.
/content/slt
--2021-01-04 17:11:46--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  21.0MB/s    in 90s     

2021-01-04 17:13:17 (20.4 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-04 17:13:17--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13

In [2]:
!cp ../drive/MyDrive/slt/late-fusion-duo/*.py signjoey/

In [3]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!unzip -q '../drive/MyDrive/slt/keypoints/train_body_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_face_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_hand_pose3d.zip' -d data

!unzip -q '../drive/MyDrive/slt/keypoints/test_keypoints.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/dev_keypoints.zip' -d data

In [4]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

## Keypoints

In [5]:
body_parts = ['body_pose3d',
              'face_pose3d',
              'hand_pose3d']

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)
  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'
    
    new_features_list = []

    for body_part in body_parts:
      dst = os.path.join(os.path.join(wd, s), os.path.join(body_part, name))

      new_features = np.load(dst)
      new_features = new_features.reshape((new_features.shape[0], np.prod(new_features.shape[1: ])))
      new_features_list.append(new_features)
      
    
    new_features = np.concatenate(new_features_list, axis=1)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [00:15<00:00, 458.04it/s]


## Change config files

In [6]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(2)]

list_doc['data']['feature_size'] = [1024, 417]

with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


## Run

In [7]:
!cp -r '../drive/MyDrive/slt/models/baseline_40' baseline
!cp -r '../drive/MyDrive/slt/models/keypoints3d_40' keypoints3d

In [8]:
import os
import sys
from signjoey.prediction import test

test(cfg_file="configs/sign.yaml", ckpts=["baseline/best.ckpt", "keypoints3d/best.ckpt"], output_path="")

2021-01-04 17:22:39,071 - ------------------------------------------------------------
2021-01-04 17:22:39,072 - [DEV] partition [RECOGNITION] experiment [BW]: 1
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)
2021-01-04 17:23:08,370 - finished in 29.2979s 
2021-01-04 17:23:08,371 - ************************************************************
2021-01-04 17:23:08,372 - [DEV] partition [RECOGNITION] results:
	New Best CTC Decode Beam Size: 1
	WER 51.69	(DEL: 48.31,	INS: 0.19,	SUB: 3.20)
2021-01-04 17:23:08,373 - ************************************************************
2021-01-04 17:23:08,374 - ------------------------------------------------------------
2021-01-04 17:23:08,376 - [DEV] partition [RECOGNITION] exper

translation


2021-01-04 17:40:48,285 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 1 and Alpha: -1
	BLEU-4 19.25	(BLEU-1: 44.83,	BLEU-2: 31.55,	BLEU-3: 23.96,	BLEU-4: 19.25)
	CHRF 42.43	ROUGE 45.05
2021-01-04 17:40:48,286 - ------------------------------------------------------------
2021-01-04 17:45:24,234 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 2 and Alpha: 2
	BLEU-4 19.29	(BLEU-1: 44.47,	BLEU-2: 31.26,	BLEU-3: 23.90,	BLEU-4: 19.29)
	CHRF 41.98	ROUGE 45.02
2021-01-04 17:45:24,235 - ------------------------------------------------------------
2021-01-04 17:48:44,461 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 3 and Alpha: 2
	BLEU-4 19.63	(BLEU-1: 44.61,	BLEU-2: 31.53,	BLEU-3: 24.20,	BLEU-4: 19.63)
	CHRF 42.05	ROUGE 45.10
2021-01-04 17:48:44,462 - ------------------------------------------------------------
2021-01-04 18:18:17,949 - ************************************************************
2021-01-04 18:1